<a href="https://colab.research.google.com/github/scmaverick/DataScienceTEC/blob/master/BigData/Tarea5/Tarea5_Supervised_Machine_Learning_Sergio_Castillo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 5 - Sergio Castillo
## Intro ML SUP en BD

Creación de la sesión Spark:

In [0]:
#Instalamos y preparamos todo el ambiente con Java y Spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
#import SparkSession
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [0]:
# Crear el spark session object, llamarle "supervised_ml"
spark = SparkSession.builder.master("supervised_ml").getOrCreate()


## Regression 

Carga de datos, archivo *Linear_regression_dataset.csv*:

In [0]:
# Carga de datos


from pyspark import SparkFiles
spark.sparkContext.addFile("https://raw.githubusercontent.com/fmezacr/BigDataTEC/master/Clase6/Linear_regression_dataset.csv")
df = spark.read.csv(SparkFiles.get("Linear_regression_dataset.csv"), header=True, inferSchema=True)

Se invocan las librerias correcpondientes a **LinearRegression**, asi como las de OneHotEncoder, StringIndexer, VectorAssembler:

In [0]:
# Importacion de libs y operaciones
import os
import numpy as np
import pandas as pd
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.sql import functions as f
from pyspark.sql.functions import udf, StringType
from pyspark.sql import SparkSession, functions as F
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.feature import OneHotEncoder, VectorAssembler, StringIndexer

Se visualizan algunos datos:

In [13]:
print((df.count(), len(df.columns))) 
df.printSchema()

(1232, 6)
root
 |-- var_1: integer (nullable = true)
 |-- var_2: integer (nullable = true)
 |-- var_3: integer (nullable = true)
 |-- var_4: double (nullable = true)
 |-- var_5: double (nullable = true)
 |-- label: double (nullable = true)



Se muestran los primeros 10 datos:

In [14]:
# primeros 10 datos
df.head(10)

[Row(var_1=734, var_2=688, var_3=81, var_4=0.328, var_5=0.259, label=0.418),
 Row(var_1=700, var_2=600, var_3=94, var_4=0.32, var_5=0.247, label=0.389),
 Row(var_1=712, var_2=705, var_3=93, var_4=0.311, var_5=0.247, label=0.417),
 Row(var_1=734, var_2=806, var_3=69, var_4=0.315, var_5=0.26, label=0.415),
 Row(var_1=613, var_2=759, var_3=61, var_4=0.302, var_5=0.24, label=0.378),
 Row(var_1=748, var_2=676, var_3=85, var_4=0.318, var_5=0.255, label=0.422),
 Row(var_1=669, var_2=588, var_3=97, var_4=0.315, var_5=0.251, label=0.411),
 Row(var_1=667, var_2=845, var_3=68, var_4=0.324, var_5=0.251, label=0.381),
 Row(var_1=758, var_2=890, var_3=64, var_4=0.33, var_5=0.274, label=0.436),
 Row(var_1=726, var_2=670, var_3=88, var_4=0.335, var_5=0.268, label=0.422)]

## Feature Engineering

Creamos un solo vector con todos los features i.e 'var_1', 'var_2', 'var_3', 'var_4', 'var_5', a este le llamaremos "features" y como salida colocamos a 'label':

In [0]:
# Vector Ensamblador
#categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
#numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

#indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]
df_assembler = []
df_assembler = VectorAssembler(inputCols=df.columns[0:5], outputCol="features")

df2 = df_assembler.transform(df).select(['features', 'label'])

In [16]:
# visulizacion de vector ensamblado compuesto por features y label

df2.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[734.0,688.0,81.0...|0.418|
|[700.0,600.0,94.0...|0.389|
|[712.0,705.0,93.0...|0.417|
|[734.0,806.0,69.0...|0.415|
|[613.0,759.0,61.0...|0.378|
|[748.0,676.0,85.0...|0.422|
|[669.0,588.0,97.0...|0.411|
|[667.0,845.0,68.0...|0.381|
|[758.0,890.0,64.0...|0.436|
|[726.0,670.0,88.0...|0.422|
|[583.0,794.0,55.0...|0.371|
|[676.0,746.0,72.0...|  0.4|
|[767.0,699.0,89.0...|0.433|
|[637.0,597.0,86.0...|0.374|
|[609.0,724.0,69.0...|0.382|
|[776.0,733.0,83.0...|0.437|
|[701.0,832.0,66.0...| 0.39|
|[650.0,709.0,74.0...|0.386|
|[804.0,668.0,95.0...|0.453|
|[713.0,614.0,94.0...|0.404|
+--------------------+-----+
only showing top 20 rows



Partimos a continuación el set de datos en 75% training y 25% testing:

In [17]:
# Particion del data set
train, test  = df2.randomSplit([0.75, 0.25], 32)

print(f"Size of train Dataset : {train.count()}" )
print(f"Size of test Dataset : {test.count()}" )

Size of train Dataset : 950
Size of test Dataset : 282


Creamos el Regresor Lineal: 

In [0]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression()

Entrenamos el modelo de regresión lineal:

In [0]:
# Fit the model, le llamamos lr_model
lr_model = lr.fit(train)



Creamos el dataframe de prediciones (*predictions_df*) a partir del modelo de entrenamiento y el conjunto de datos test: 

In [0]:
predictions_df = lr_model.transform(test)

Visualizamos el contenido de *predictions_df*:

In [21]:
# visulizacion de predictions_df
predictions_df.show()


+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[464.0,640.0,66.0...|0.301|0.31425090959414237|
|[486.0,610.0,61.0...|0.332| 0.3200618835372234|
|[510.0,588.0,72.0...|0.317|0.32409714928176037|
|[527.0,569.0,75.0...|0.341| 0.3342283294791447|
|[543.0,615.0,76.0...|0.333|   0.34077679575515|
|[544.0,551.0,82.0...|0.344|0.33687681741528375|
|[545.0,661.0,61.0...| 0.35|0.34062557118153797|
|[548.0,636.0,63.0...|0.339|0.33720156902021947|
|[550.0,789.0,54.0...|0.359| 0.3439135683303223|
|[557.0,659.0,71.0...|0.355|0.34689693716060926|
|[570.0,578.0,82.0...|0.363| 0.3496399088079572|
|[573.0,656.0,75.0...|0.345|0.34480544862021006|
|[574.0,517.0,83.0...|0.331| 0.3448039125603529|
|[576.0,759.0,57.0...| 0.35|0.35479335399043554|
|[583.0,532.0,80.0...|0.343|0.35312604537815406|
|[586.0,688.0,69.0...|0.338|0.35173641672617584|
|[586.0,745.0,64.0...|0.349| 0.3581752775912789|
|[587.0,663.0,64.0..

Ahora, evaluamos el modelo de Regresión Lineal, con los datos de TEST:

In [0]:
# evaluacion del modelo, le llamaremos model_predictions

model_predictions = lr_model.evaluate(test)


Imprimimos el valor de R2:

In [23]:
# valor de R2
print("r2: %f" % lr_model.summary.r2)



r2: 0.865380


Imprimimos el valor del meanSquaredError:

In [24]:
# valor del meanSquaredError
print("MSE: %f" % lr_model.summary.meanSquaredError)



MSE: 0.000145


## Regresión con Árboles de Decisión

Importamos la librería *DecisionTreeRegressor*: 

In [0]:
# import lib

from pyspark.ml.regression import DecisionTreeRegressor


Creamos el Regresor DT, le llamaremos *dec_tree*:

In [0]:
# dec_tree

dec_tree = DecisionTreeRegressor()


Entrenamos el modelo:

In [0]:
# Train model, le llamaremos dec_tree_model

dec_tree_model = dec_tree.fit(train)

In [28]:
dec_tree.getMaxDepth()

5

Cuánto es la profundidad máxima por defecto, de este algoritmo?

R/ 5

Desplegamos las *featureImportances*:

In [29]:
dec_tree_model.featureImportances

SparseVector(5, {0: 0.9622, 1: 0.0169, 2: 0.005, 3: 0.0049, 4: 0.0109})

Evaluamos el modelo con los datos de entrenamiento:

In [0]:
# Make predictions, le llamaremos model_predictions 
model_predictions = dec_tree_model.transform(test)



In [31]:
# visualizamos

model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[464.0,640.0,66.0...|0.301|0.32023076923076926|
|[486.0,610.0,61.0...|0.332|0.32023076923076926|
|[510.0,588.0,72.0...|0.317|0.32023076923076926|
|[527.0,569.0,75.0...|0.341|             0.3285|
|[543.0,615.0,76.0...|0.333|0.32023076923076926|
|[544.0,551.0,82.0...|0.344|             0.3285|
|[545.0,661.0,61.0...| 0.35|             0.3285|
|[548.0,636.0,63.0...|0.339|             0.3285|
|[550.0,789.0,54.0...|0.359|0.34299999999999997|
|[557.0,659.0,71.0...|0.355|  0.351974025974026|
|[570.0,578.0,82.0...|0.363|  0.351974025974026|
|[573.0,656.0,75.0...|0.345|  0.351974025974026|
|[574.0,517.0,83.0...|0.331|  0.351974025974026|
|[576.0,759.0,57.0...| 0.35|  0.351974025974026|
|[583.0,532.0,80.0...|0.343|  0.351974025974026|
|[586.0,688.0,69.0...|0.338|  0.351974025974026|
|[586.0,745.0,64.0...|0.349|  0.351974025974026|
|[587.0,663.0,64.0..

Importamos el **RegressionEvaluator**

In [0]:
# import Evaluator

from pyspark.ml.evaluation import RegressionEvaluator


Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [33]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of DecisionTreeRegressor is {dt_r2}')

# RMSE value of the model on test data 
dt_evaluator.setMetricName("rmse")
rmse = dt_evaluator.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))



The r-square value of DecisionTreeRegressor is 0.8467659356512574
Root Mean Squared Error (RMSE) on test data = 0.013504505717714619


## RandomForestRegressor

Importamos a *RandomForestRegressor*

In [0]:
# import lib
from pyspark.ml.regression import RandomForestRegressor



Creamos el Regresor RF:

In [0]:
# Regresor 

rf = RandomForestRegressor()


Entrenamos el modelo:

In [0]:
# Train model, le llamaremos rf_model
rf_model = rf.fit(train)


Desplegamos las *featureImportances*:

In [37]:
# importances 

rf_model.featureImportances


SparseVector(5, {0: 0.5294, 1: 0.0409, 2: 0.0149, 3: 0.2028, 4: 0.2121})

Desplegamos el numero de arboles (Num of Trees)

In [38]:
# Numero de Trees

print("Trees created: " + str(rf_model.getNumTrees))


Trees created: 20


Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:


In [0]:
# model_predictions

model_predictions = rf_model.transform(train)


Desplegamos los valores del *model_predictions*

In [40]:
model_predictions.show()

+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[463.0,527.0,67.0...|0.311|   0.32013779169502|
|[468.0,746.0,52.0...|0.329| 0.3304187013228318|
|[470.0,509.0,76.0...|0.319| 0.3202839333311617|
|[473.0,499.0,73.0...|0.315|0.32009393333116165|
|[495.0,628.0,66.0...|0.315|0.32502236381372257|
|[495.0,752.0,50.0...|0.327| 0.3304187013228318|
|[498.0,615.0,67.0...|0.318|0.32447403048038925|
|[498.0,672.0,61.0...|0.325|0.33468477932603913|
|[501.0,774.0,51.0...|0.315| 0.3304187013228318|
|[511.0,576.0,76.0...|0.329|0.32366359602957434|
|[513.0,698.0,61.0...|0.339|0.34153971142825046|
|[514.0,549.0,81.0...|0.339|0.35338306061506897|
|[516.0,504.0,86.0...|0.327| 0.3220756320491104|
|[519.0,595.0,73.0...|0.332| 0.3359821210841545|
|[522.0,621.0,72.0...|0.317| 0.3246227911641499|
|[524.0,665.0,65.0...|0.336|0.32827497269035316|
|[528.0,652.0,71.0...|0.319| 0.3310317805556903|
|[531.0,491.0,89.0..

Usando *RegressionEvaluator* calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [41]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of RandomForestRegressor is {dt_r2}')

# RMSE value of the model on test data 
dt_evaluator.setMetricName("rmse")
rmse = dt_evaluator.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

The r-square value of RandomForestRegressor is 0.8654055169547652
Root Mean Squared Error (RMSE) on test data = 0.01204844436520372


## Gradient-Boosted Tree Regressor

Importamos a GBTRegressor


In [0]:
# import
from pyspark.ml.regression import GBTRegressor



Creamos el Regresor GBTR, le llamaremos gbt:


In [0]:
# regresor

gbt = GBTRegressor()


Entrenamos el modelo:

In [0]:
# Train model, le llamaremos gbt_model


gbt_model = gbt.fit(train)



Desplegamos las featureImportances:

In [45]:
#Importances
gbt_model.featureImportances



SparseVector(5, {0: 0.2346, 1: 0.2108, 2: 0.1888, 3: 0.2032, 4: 0.1626})

Evaluamos el modelo con los datos de entrenamiento, le llamaremos model_predictions:

In [0]:
# Model
model_predictions = gbt_model.transform(train)


Desplegamos los valores del *model_predictions*

In [47]:
# show 

model_predictions.show()


+--------------------+-----+-------------------+
|            features|label|         prediction|
+--------------------+-----+-------------------+
|[463.0,527.0,67.0...|0.311| 0.3164344441725342|
|[468.0,746.0,52.0...|0.329| 0.3266866071370786|
|[470.0,509.0,76.0...|0.319|0.32096605235456055|
|[473.0,499.0,73.0...|0.315| 0.3176814283351136|
|[495.0,628.0,66.0...|0.315| 0.3174545939258208|
|[495.0,752.0,50.0...|0.327| 0.3266866071370786|
|[498.0,615.0,67.0...|0.318| 0.3187126978489555|
|[498.0,672.0,61.0...|0.325|0.32629949170524275|
|[501.0,774.0,51.0...|0.315|  0.324696547012342|
|[511.0,576.0,76.0...|0.329| 0.3221578747828934|
|[513.0,698.0,61.0...|0.339| 0.3392956804388739|
|[514.0,549.0,81.0...|0.339|0.33402954670576834|
|[516.0,504.0,86.0...|0.327|0.32174698115061107|
|[519.0,595.0,73.0...|0.332|0.32754215771195183|
|[522.0,621.0,72.0...|0.317| 0.3180148230195864|
|[524.0,665.0,65.0...|0.336| 0.3253709202766713|
|[528.0,652.0,71.0...|0.319| 0.3185904900297776|
|[531.0,491.0,89.0..

Usando RegressionEvaluator calculamos e imprimimos el valor de las metricas R2 y RMSE:

In [48]:
# R2 value of the model on test data 
dt_evaluator = RegressionEvaluator(metricName='r2')
dt_r2 = dt_evaluator.evaluate(model_predictions)
print(f'The r-square value of RandomForestRegressor is {dt_r2}')

# RMSE value of the model on test data 
dt_evaluator.setMetricName("rmse")
rmse = dt_evaluator.evaluate(model_predictions)
print("Root Mean Squared Error (RMSE) on test data = " + str(rmse))

The r-square value of RandomForestRegressor is 0.9239581523452711
Root Mean Squared Error (RMSE) on test data = 0.009056151466607818


 ## Exploracion de datos...

Usaremos el dataset https://archive.ics.uci.edu/ml/datasets/Bank+Marketing 

Indique a grandes razgos de que se trata este dataset:


Se trata de llamadas de telemercadeo, con las variables indicadas se trata de determinar si el cliente obtendrá el producto o no.

Carga de datos, archivo bank_data.csv:


In [0]:
# Load csv Dataset 
spark.sparkContext.addFile("https://raw.githubusercontent.com/fmezacr/BigDataTEC/master/Clase6/bank_data.csv")
df=spark.read.csv(SparkFiles.get('bank_data.csv'),inferSchema=True,header=True)

Determine la cantidad de datos en el dataset:

In [50]:
#number of records
print("Number of records:", df.count())




Number of records: 41188


A que dato corresponde cada columna?

In [51]:
# columns values
df.columns



['age',
 'job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'month',
 'day_of_week',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'emp.var.rate',
 'cons.price.idx',
 'cons.conf.idx',
 'euribor3m',
 'nr.employed',
 'target_class']

Imprima el Schema:

In [52]:
#dataype of input data - Schema

df.printSchema()



root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day_of_week: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- emp.var.rate: double (nullable = true)
 |-- cons.price.idx: double (nullable = true)
 |-- cons.conf.idx: double (nullable = true)
 |-- euribor3m: double (nullable = true)
 |-- nr.employed: double (nullable = true)
 |-- target_class: string (nullable = true)



En cuanto a la salida, como es la distrubución de clases?

In [70]:
# YES/NO Class Distribution
print("Has credit in default:")
df.groupBy('default').count().show()
print("Has housing loan:")
df.groupBy('housing').count().show()
print("Has personal loan:")
df.groupBy('loan').count().show()
print("Has the client subscribed a term deposit:")
df.groupBy('target_class').count().show()

Has credit in default:
+-------+-----+
|default|count|
+-------+-----+
|unknown| 8597|
|     no|32588|
|    yes|    3|
+-------+-----+

Has housing loan:
+-------+-----+
|housing|count|
+-------+-----+
|unknown|  990|
|     no|18622|
|    yes|21576|
+-------+-----+

Has personal loan:
+-------+-----+
|   loan|count|
+-------+-----+
|unknown|  990|
|     no|33950|
|    yes| 6248|
+-------+-----+

Has the client subscribed a term deposit:
+------------+-----+
|target_class|count|
+------------+-----+
|          no|36548|
|         yes| 4640|
+------------+-----+



Una tarea típica, resulta de convertir los valores binarios en 1 y 0, usando como referencia "label", convierta los no/yes en 0/1:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import *

In [0]:
# Ingrese acá la instrucción: 
stringIndexer = StringIndexer(inputCol="target_class", outputCol="target_class_idx")
model = stringIndexer.fit(df)
indexed = model.transform(df)
encoder = OneHotEncoder(dropLast=False, inputCol="target_class_idx", outputCol="tgtVector")
encoded = encoder.transform(indexed)


In [85]:
# New 1/0 Class Distribution
encoded.show()
encoded.createOrReplaceTempView("table1")
df2 = spark.sql("select * from table1 where target_class='yes'")
df2.show()

+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+----------------+-------------+
|age|        job| marital|          education|default|housing|loan|  contact|month|day_of_week|duration|campaign|pdays|previous|   poutcome|emp.var.rate|cons.price.idx|cons.conf.idx|euribor3m|nr.employed|target_class|target_class_idx|    tgtVector|
+---+-----------+--------+-------------------+-------+-------+----+---------+-----+-----------+--------+--------+-----+--------+-----------+------------+--------------+-------------+---------+-----------+------------+----------------+-------------+
| 56|  housemaid| married|           basic.4y|     no|     no|  no|telephone|  may|        mon|     261|       1|  999|       0|nonexistent|         1.1|        93.994|        -36.4|    4.857|     5191.0|          no|             0.0|(2,[0],[1.0])|
| 57

A continuación se presenta un ejercicio de Deep Learning para su revisión...

# Deep Learning 

Importamos las librerias necesarias:

Inicializamos la sesion SPARK:

In [0]:
spark = SparkSession.builder.appName('deep_learning').getOrCreate()

Leemos el dataset:

In [0]:
spark.sparkContext.addFile("https://raw.githubusercontent.com/fmezacr/BigDataTEC/master/Clase6/dl_data.csv")
data = spark.read.csv(SparkFiles.get('dl_data.csv'), header=True, inferSchema=True)

In [89]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- Orders_Normalized: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Renombramos la columna TARGET:

In [0]:
data = data.withColumnRenamed('Orders_Normalized', 'label')

In [91]:
data.printSchema()

root
 |-- Visit_Number_Bucket: string (nullable = true)
 |-- Page_Views_Normalized: double (nullable = true)
 |-- label: integer (nullable = true)
 |-- Internal_Search_Successful_Normalized: double (nullable = true)
 |-- Internal_Search_Null_Normalized: double (nullable = true)
 |-- Email_Signup_Normalized: double (nullable = true)
 |-- Total_Seconds_Spent_Normalized: double (nullable = true)
 |-- Store_Locator_Search_Normalized: double (nullable = true)
 |-- Mapped_Last_Touch_Channel: string (nullable = true)
 |-- Mapped_Mobile_Device_Type: string (nullable = true)
 |-- Mapped_Browser_Type: string (nullable = true)
 |-- Mapped_Entry_Pages: string (nullable = true)
 |-- Mapped_Site_Section: string (nullable = true)
 |-- Mapped_Promo_Code: string (nullable = true)
 |-- Maped_Product_Name: string (nullable = true)
 |-- Mapped_Search_Term: string (nullable = true)
 |-- Mapped_Product_Collection: string (nullable = true)



Partimos lo datos en Train, Validation y Test:

In [0]:
train, validation, test  = data.randomSplit([0.7, 0.2, 0.1], 1234)

Construimos el Pipeline

In [0]:
categorical_columns = [item[0] for item in data.dtypes if item[1].startswith('string')]
numeric_columns = [item[0] for item in data.dtypes if item[1].startswith('double')]

indexers = [StringIndexer(inputCol=column, outputCol='{0}_index'.format(column)) for column in categorical_columns]

featuresCreator = VectorAssembler(inputCols=[indexer.getOutputCol() for indexer in indexers] + numeric_columns, outputCol="features")

layers = [len(featuresCreator.getInputCols()), 4, 2, 2]

classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=100, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])

Entrenamos...

In [0]:
model = pipeline.fit(train)

Validamos y Evaluamos

In [0]:
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)

Llevamos a cabo, algunas predicciones:

In [96]:
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")

metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']

for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9722605697126978
Validation weightedPrecision = 0.9734944186485901
Test weightedPrecision = 0.9710090865749514
Train weightedRecall = 0.9718655625913297
Validation weightedRecall = 0.9731379731379731
Test weightedRecall = 0.9706199460916443
Train accuracy = 0.9718655625913297
Validation accuracy = 0.9731379731379731
Test accuracy = 0.9706199460916443


Puede mejorar el test accuracy del modelo variando alguno de los hyperparametros?

In [98]:
#Cambiamos maxIter a 300
classifier = MultilayerPerceptronClassifier(labelCol='label', featuresCol='features', maxIter=300, layers=layers, blockSize=128, seed=1234)

pipeline = Pipeline(stages=indexers + [featuresCreator, classifier])
model = pipeline.fit(train)
train_output_df = model.transform(train)
validation_output_df = model.transform(validation)
test_output_df = model.transform(test)
train_predictionAndLabels = train_output_df.select("prediction", "label")
validation_predictionAndLabels = validation_output_df.select("prediction", "label")
test_predictionAndLabels = test_output_df.select("prediction", "label")
metrics = ['weightedPrecision', 'weightedRecall', 'accuracy']
for metric in metrics:
    evaluator = MulticlassClassificationEvaluator(metricName=metric)
    print('Train ' + metric + ' = ' + str(evaluator.evaluate(train_predictionAndLabels)))
    print('Validation ' + metric + ' = ' + str(evaluator.evaluate(validation_predictionAndLabels)))
    print('Test ' + metric + ' = ' + str(evaluator.evaluate(test_predictionAndLabels)))

Train weightedPrecision = 0.9750134646991298
Validation weightedPrecision = 0.9761981207517723
Test weightedPrecision = 0.9729149285584404
Train weightedRecall = 0.9743399902581589
Validation weightedRecall = 0.9755799755799756
Test weightedRecall = 0.9722371967654986
Train accuracy = 0.9743399902581588
Validation accuracy = 0.9755799755799756
Test accuracy = 0.9722371967654987


R/ Con solo aumentar la cantidad de iteraciones se logró aumentar la precisión de test accuracy de 97.06% a 97.22%, por lo tanto se podría seguir aumentando las iteraciones o algún otro parámetro y de seguro se podría obtener un mejor valor.